In [9]:
# export
from fastai.vision.all import *
import torch.nn as nn
import torch.nn.functional as F

In [34]:
# export
# Data Preparation
path = untar_data(URLs.MNIST)

dls = ImageDataLoaders.from_folder(path, train='training', valid='testing', convert_mode='L')

In [ ]:
dls.show_batch()

In [36]:
# export
# Model Definition
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3*28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.layers(x)

In [37]:
# export
# Loss and Optimizer
device = torch.device("cuda:0")
model = MyModel().to(device)
opt = SGD(model.parameters(), lr=0.01)

In [38]:
for xb, yb in dls.train:
    print(xb.shape)
    break

torch.Size([64, 3, 28, 28])


In [39]:
# export
# Training Loop
epochs = 10  # Number of epochs
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    for xb, yb in dls.train:
        preds = model(xb)
        loss = F.cross_entropy(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()

    # Validation Loop
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No gradient calculation needed
        val_loss = sum(F.cross_entropy(model(xb), yb) for xb, yb in dls.valid)
        val_loss /= len(dls.valid)
        
    # Evaluate training and validation accuracy
    def batch_accuracy(xb, yb):
        preds = xb.argmax(dim=1)
        return (preds == yb).float().mean()
    
    train_acc = sum(batch_accuracy(model(xb), yb) for xb, yb in dls.train) / len(dls.train)
    valid_acc = sum(batch_accuracy(model(xb), yb) for xb, yb in dls.valid) / len(dls.valid)
    
    print(f"Epoch: {epoch}, Training Loss: {loss}, Validation Loss: {val_loss}, Training Acc: {train_acc}, Validation Acc: {valid_acc}")

TypeError: no implementation found for 'torch.nn.functional.cross_entropy' on types that implement __torch_function__: [<class 'fastai.torch_core.TensorImage'>, <class 'fastai.torch_core.TensorCategory'>]

In [40]:
import nbformat

def export_cells(notebook_path, export_path, directive="# export"):
    with open(notebook_path, 'r') as f:
        notebook = nbformat.read(f, as_version=4)

    script_lines = []
    script_lines.append("# Generated by the corresponding notebook.")
    for cell in notebook.cells:
        if cell.cell_type == "code":
            if cell.source.startswith(directive):
                # Remove the first line (the directive itself)
                cleaned_source = "\n".join(cell.source.split("\n")[1:])
                script_lines.append(cleaned_source)

    with open(export_path, 'w') as f:
        f.write("\n\n".join(script_lines))

# Usage
export_cells("nn.ipynb", "nn.py")
